In [4]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import csv
import copy
import ast
import os
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection


In [5]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [6]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
def zip_code_geojson_filename(state_fips_code=1):
    formatted_fips_code = formatted_state_fips_code(state_fips_code)
    state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
    state_full_name = fips_codes[formatted_fips_code]["full_name"]
    return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


def zip_geojson_files_for_market(market_data):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(zip_code_geojson_filename(fips_code))
    return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "zip"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    zip_codes = []
    for data_row in data_rows:
        zip_codes.append(data_row['Zip'].zfill(5))
    print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
    return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return zip_codes
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "zip"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "zip"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "zip"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_data.json'

def published_map_settings_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data.json'

def published_map_labels_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_labels.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='zip'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, bg_data_rows):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        for feature in features:
            if feature['properties']["GEOID10"] == block_group:
                found = True
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'BG':
                        if data_row[key] and data_row[key] != 'inf':
                            new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)

def write_geojson(geojson_data, market_prefix, data_type='zip',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{data_type}_data.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}_{data_type}_data.json')
    
def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "zip"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, base_path="./public", data_type = "zip"):
    processed_settings_data = OrderedDict()
    for row in settings_data:
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        processed_settings_data[row['UniqueSeriesString']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    
def create_zip_labels(market_file_prefix, base_path="./source_data" , data_type = "zip" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "zip")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['Zip']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['intptlong']), float(data_row['intptlat']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_prefix,  data_type , published_data_base_path="./public")
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_bg_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_prefix,  data_type , published_data_base_path="./public")
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings.json')
    

In [7]:
def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            print(market_data)
            market_file_prefix = market_data['MarketName']
            market_prefix = market_file_prefix
            # Zip Processing     
            data_type = 'zip'
            print(f'market: {market_prefix}')
            geojson_files = zip_geojson_files_for_market(market_data)
            zip_codes = read_market_zip_code_usage(market_file_prefix, base_path=source_data_base_path )
            zip_data_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
            features_to_use = create_zip_features(features, zip_data_rows)
            new_geojson = create_geojson(features_to_use)
            write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
            map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "zip")
            write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "zip")
            create_zip_labels(market_file_prefix, base_path=source_data_base_path , data_type = "zip" , published_data_base_path=published_data_base_path)
            # Block Group Processing
            data_type = 'bg'
            geojson_files = bg_geojson_files_for_market(market_data,census_year )
            block_groups = read_market_block_group_usage(market_file_prefix, base_path=source_data_base_path )
            block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
            features_to_use = create_bg_features(features, block_group_rows)
            new_geojson = create_geojson(features_to_use)
            write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
            map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "bg")
            write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "bg")
            create_bg_labels(market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    create_geojson_settings(csv_file_path, published_data_base_path = "./public")
    print(f'*** COMPLETED {market_count} markets')
    print(completed_markets)
                          
        
source_data_base_path = "./source_data"
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie/"
published_data_base_path = "./public"
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
census_year = 2010



Market Data: 1
{'MarketName': 'Albany', 'Market_Display_Name': 'Albany', 'StateFips': '[9, 25, 36, 50]', 'CenterLat': '42.675748', 'CenterLong': '-73.74473499999999', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: Albany
Loaded 201 zip_codes from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Albany__Mapping_Zip_Data.csv
Loaded 201 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Albany__Mapping_Zip_Data.csv
Loaded 282 features from /Users/alanmccann/Dropbox/bain/map_source_data/ct_connecticut_zip_codes_geo.min.json
Loaded 538 features from /Users/alanmccann/Dropbox/bain/map_source_data/ma_massachusetts_zip_codes_geo.min.json
Loaded 1794 features from /Users/alanmccann/Dropbox/bain/map_source_data/ny_new_york_zip_codes_geo.min.json
Loaded 255 features from /Users/alanmccann/Dropbox/bain/map_source_data/vt_vermont_zip_codes_geo.min.json
Processed 201 features to use
Created GeoJson with  201 featur

Loaded 68 features from /Users/alanmccann/Dropbox/bain/map_source_data/de_delaware_zip_codes_geo.min.json
Loaded 53 features from /Users/alanmccann/Dropbox/bain/map_source_data/dc_district_of_columbia_zip_codes_geo.min.json
Loaded 468 features from /Users/alanmccann/Dropbox/bain/map_source_data/md_maryland_zip_codes_geo.min.json
Loaded 1798 features from /Users/alanmccann/Dropbox/bain/map_source_data/pa_pennsylvania_zip_codes_geo.min.json
Processed 310 features to use
Created GeoJson with  310 features
Wrote geojson data to  ./public/Baltimore_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Baltimore__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/Baltimore_zip_settings_data.json
Wrote geojson labels data to  ./public/Baltimore_zip_labels.json
Loaded 3656 block groups from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Baltimore__Mapping_BG_Data.csv
Loaded 3656 data rows fro

Processed 4108 features to use
Created GeoJson with  4108 features
Wrote geojson data to  ./public/Boston_bg_data.json
Loaded 172 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Boston__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Boston_bg_settings_data.json
Wrote geojson labels data to  ./public/Boston_bg_labels.json

Market Data: 9
{'MarketName': 'Buffalo', 'Market_Display_Name': 'Buffalo', 'StateFips': '[36]', 'CenterLat': '42.908938', 'CenterLong': '-78.820824', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: Buffalo
Loaded 122 zip_codes from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Buffalo__Mapping_Zip_Data.csv
Loaded 122 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Buffalo__Mapping_Zip_Data.csv
Loaded 1794 features from /Users/alanmccann/Dropbox/bain/map_source_data/ny_new_york_zip_codes_geo.min.json
Processed 122 f

Loaded 424 features from /Users/alanmccann/Dropbox/bain/map_source_data/sc_south_carolina_zip_codes_geo.min.json
Processed 44 features to use
Created GeoJson with  44 features
Wrote geojson data to  ./public/CharlestonSC_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//CharlestonSC__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/CharlestonSC_zip_settings_data.json
Wrote geojson labels data to  ./public/CharlestonSC_zip_labels.json
Loaded 397 block groups from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//CharlestonSC__Mapping_BG_Data.csv
Loaded 397 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//CharlestonSC__Mapping_BG_Data.csv
Loaded 3059 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_45_bg10.json
Processed 397 features to use
Created GeoJson with  397 features
Wrote geojson data to  ./public/CharlestonSC_bg_data.js

Loaded 776 features from /Users/alanmccann/Dropbox/bain/map_source_data/in_indiana_zip_codes_geo.min.json
Loaded 770 features from /Users/alanmccann/Dropbox/bain/map_source_data/ky_kentucky_zip_codes_geo.min.json
Loaded 1197 features from /Users/alanmccann/Dropbox/bain/map_source_data/oh_ohio_zip_codes_geo.min.json
Processed 173 features to use
Created GeoJson with  173 features
Wrote geojson data to  ./public/Cincinnati_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Cincinnati__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/Cincinnati_zip_settings_data.json
Wrote geojson labels data to  ./public/Cincinnati_zip_labels.json
Loaded 1657 block groups from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Cincinnati__Mapping_BG_Data.csv
Loaded 1657 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Cincinnati__Mapping_BG_Data.csv
Loaded 4814 featur

...

Loaded 248 features from /Users/alanmccann/Dropbox/bain/map_source_data/nh_new_hampshire_zip_codes_geo.min.json
Processed 242 features to use
Created GeoJson with  242 features
Wrote geojson data to  ./public/NewHampshire_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//NewHampshire__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/NewHampshire_zip_settings_data.json
Wrote geojson labels data to  ./public/NewHampshire_zip_labels.json
Loaded 922 block groups from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//NewHampshire__Mapping_BG_Data.csv
Loaded 922 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//NewHampshire__Mapping_BG_Data.csv
Loaded 922 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_33_bg10.json
Processed 922 features to use
Created GeoJson with  922 features
Wrote geojson data to  ./public/NewHampshire_bg_data.js

Wrote geojson labels data to  ./public/NorthCarolina_bg_labels.json

Market Data: 131
{'MarketName': 'NorthDakota', 'Market_Display_Name': 'North Dakota', 'StateFips': '[38]', 'CenterLat': '47.4501', 'CenterLong': '-100.4659', 'GeoTypeGroup': '2 - States', 'DefaultZoomLevel': '6'}
market: NorthDakota
Loaded 380 zip_codes from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//NorthDakota__Mapping_Zip_Data.csv
Loaded 380 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//NorthDakota__Mapping_Zip_Data.csv
Loaded 393 features from /Users/alanmccann/Dropbox/bain/map_source_data/nd_north_dakota_zip_codes_geo.min.json
Processed 380 features to use
Created GeoJson with  380 features
Wrote geojson data to  ./public/NorthDakota_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//NorthDakota__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/NorthDakota_zip_setti

Loaded 9740 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_42_bg10.json
Processed 9740 features to use
Created GeoJson with  9740 features
Wrote geojson data to  ./public/Pennsylvania_bg_data.json
Loaded 172 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Pennsylvania__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Pennsylvania_bg_settings_data.json
Wrote geojson labels data to  ./public/Pennsylvania_bg_labels.json

Market Data: 136
{'MarketName': 'RhodeIsland', 'Market_Display_Name': 'Rhode Island', 'StateFips': '[44]', 'CenterLat': '41.6762', 'CenterLong': '-71.5562', 'GeoTypeGroup': '2 - States', 'DefaultZoomLevel': '6'}
market: RhodeIsland
Loaded 77 zip_codes from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//RhodeIsland__Mapping_Zip_Data.csv
Loaded 77 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//RhodeIsland__Mapping_Zip_Data.csv
Load

Wrote geojson data to  ./public/Texas_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Texas__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/Texas_zip_settings_data.json
Wrote geojson labels data to  ./public/Texas_zip_labels.json
Loaded 15811 block groups from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Texas__Mapping_BG_Data.csv
Loaded 15811 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Texas__Mapping_BG_Data.csv
Loaded 15811 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_48_bg10.json
Processed 15811 features to use
Created GeoJson with  15811 features
Wrote geojson data to  ./public/Texas_bg_data.json
Loaded 172 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//Texas__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Texas_bg_settings_data.json
Wrote 

Loaded 708 features from /Users/alanmccann/Dropbox/bain/map_source_data/wv_west_virginia_zip_codes_geo.min.json
Processed 678 features to use
Created GeoJson with  678 features
Wrote geojson data to  ./public/WestVirginia_zip_data.json
Loaded 78 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//WestVirginia__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/WestVirginia_zip_settings_data.json
Wrote geojson labels data to  ./public/WestVirginia_zip_labels.json
Loaded 1592 block groups from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//WestVirginia__Mapping_BG_Data.csv
Loaded 1592 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie//WestVirginia__Mapping_BG_Data.csv
Loaded 1592 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_54_bg10.json
Processed 1592 features to use
Created GeoJson with  1592 features
Wrote geojson data to  ./public/WestVirginia_bg_da

In [8]:

#  Must run
def create_chart_json(csv_file_path, published_data_base_path = "./public"):

    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts.json')
source_data_base_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie'
csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
create_chart_json(csv_file_path)

Albany 2012
0.34459764362252016 0.30395997768333305 0.3514423786941468
Albany 2012
0.3853360465837284 0.2904436840829533 0.3242202693333183
Albany 2012
0.3262954112344596 0.3162426772336328 0.3574619115319076
Albany 2012
0.3124189204906373 0.30361536328093547 0.38396571622842723
Albany 2012
0.3452699555173957 0.26935267306655947 0.3853773714160448
Albany 2012
0.3324593049924995 0.32255365046304907 0.34498704454445145
Albany 2012
0.3614840923501548 0.3466487635671926 0.29186714408265263
Albany 2012
0.4105042341252477 0.24521891328408535 0.3442768525906669
Albany 2012
0.3183154905909111 0.276362108678961 0.40532240073012793
Albany 2012
0.30085420387368306 0.2886156913236699 0.41053010480264707
Albany 2012
0.3597202577400597 0.3141442715700141 0.32613547068992615
Albany 2012
0.3114210112279196 0.3249803332618179 0.36359865551026244
Albany 2012
0.3229650430212046 0.33570939986319615 0.34132555711559925
Albany 2012
0.37630652360753003 0.3460589425822729 0.27763453381019715
Albany 2012
0.353

0.5109537565527282 0.38475086055070074 0.10429538289657099
Austin 2016
0.39850689884689416 0.4598067963086222 0.1416863048444836
Austin 2016
0.351609336940597 0.43190990229828236 0.21648076076112066
Austin 2016
0.4952773978631495 0.34436402291223744 0.16035857922461305
Austin 2016
0.4173552056767259 0.44600835096583186 0.13663644335744227
Austin 2016
0.4200335502455895 0.48311491174732957 0.0968515380070809
Austin 2016
0.5697989012458065 0.29733841756478896 0.13286268118940459
Austin 2016
0.44224714908700824 0.39610783330612626 0.1616450176068655
Austin 2016
0.3590285311863568 0.4005024838691258 0.2404689849445174
Austin 2016
0.47858340451108794 0.4241166049792794 0.09729999050963273
Austin 2016
0.3627435324177579 0.4719852169548752 0.1652712506273669
Austin 2016
0.3316802969665333 0.4602401252827562 0.2080795777507105
Austin 2016
0.4610141555167733 0.4696238122939694 0.06936203218925732
Austin 2016
0.39261350045448024 0.5018609768932689 0.10552552265225087
Austin 2016
0.36388017824723

0.587519519165611 0.23592410359152646 0.1765563772428626
Baton Rouge 2016
0.4502063120330099 0.30244228839076615 0.24735139957622393
Baton Rouge 2016
0.41558966982086265 0.3183789979995954 0.26603133217954195
Baton Rouge 2016
0.44772590490929415 0.388487662281833 0.16378643280887284
Baton Rouge 2016
0.411055178619197 0.38408599012669925 0.20485883125410373
Baton Rouge 2016
0.4845082262519515 0.3361354629518434 0.17935631079620512
Baton Rouge 2016
0.4191951050274396 0.45887213776572255 0.12193275720683783
Baton Rouge 2016
0.46528982079983183 0.37245782752640705 0.1622523516737611
Baton Rouge 2016
0.6125482851457333 0.24206953060985603 0.14538218424441063
Baton Rouge 2019
0.4612379307773067 0.3439448788631552 0.19481719035953807
Baton Rouge 2019
0.49517920027263435 0.34211348403953196 0.1627073156878337
Baton Rouge 2019
0.4284863659959083 0.3662478243610492 0.20526580964304253
Baton Rouge 2019
0.45485215579002214 0.309248404324606 0.2358994398853719
Baton Rouge 2019
0.48635594032520846 0

0.4441526676320311 0.3045049506188586 0.2513423817491103
Boston 2021
0.27312374742005924 0.3832510743521482 0.3436251782277926
Boston 2021
0.33365634406828587 0.37510480667463886 0.2912388492570753
Buffalo 2012
0.5781976952258249 0.29537500612170325 0.12642729865247185
Buffalo 2012
0.6082757142091441 0.27053116792624754 0.12119311786460846
Buffalo 2012
0.5505859987732187 0.3167905153886134 0.13262348583816788
Buffalo 2012
0.5803794709431711 0.29584509142634274 0.12377543763048615
Buffalo 2012
0.6192706904765342 0.24973293297339436 0.13099637655007146
Buffalo 2012
0.5456481698134715 0.3175363127486096 0.13681551743791884
Buffalo 2012
0.5098523015481369 0.3981202015217416 0.09202749693012154
Buffalo 2012
0.6632051079098729 0.20987810669621657 0.1269167853939106
Buffalo 2012
0.6108109944619985 0.25402326187159535 0.13516574366640619
Buffalo 2012
0.5829416143783899 0.28586621291077313 0.13119217271083697
Buffalo 2012
0.5720162677928985 0.29623025183794777 0.13175348036915377
Buffalo 2012
0

0.3165684941851684 0.3911997180687711 0.2922317877460605
CharlestonWV 2012
0.3563040791100124 0.3978162340337866 0.24587968685620107
CharlestonWV 2012
0.35904304061579234 0.331058886554918 0.3098980728292896
CharlestonWV 2012
0.2854200262897565 0.35567059206157764 0.35890938164866587
CharlestonWV 2012
0.3015870760574587 0.3624557764169307 0.3359571475256106
CharlestonWV 2012
0.3117957228125586 0.40561012820855974 0.28259414897888174
CharlestonWV 2012
0.30177320490367776 0.3861865148861646 0.3120402802101576
CharlestonWV 2012
0.3690549342509524 0.3721887673589775 0.25875629839007
CharlestonWV 2012
0.30709472849782005 0.4394768133174792 0.2534284581847008
CharlestonWV 2012
0.3560306868219166 0.39652276544554904 0.24744654773253438
CharlestonWV 2012
0.5059694364851958 0.2791308500477555 0.2148997134670487
CharlestonWV 2016
0.3219976130304496 0.36660875737341764 0.31139362959613276
CharlestonWV 2016
0.33975568051613814 0.3608383719085477 0.2994059475753142
CharlestonWV 2016
0.3075839714645

...

SouthBendElkhart 2016
0.3347360893363039 0.2588222350578981 0.4064416756057981
SouthBendElkhart 2016
0.2966777001348535 0.27399779330636265 0.42932450655878385
SouthBendElkhart 2019
0.3448047842926383 0.27105905035957284 0.38413616534778894
SouthBendElkhart 2019
0.37015939744263443 0.2634296724470135 0.3664109301103521
SouthBendElkhart 2019
0.3369625993081456 0.2764406977530092 0.38659670293884524
SouthBendElkhart 2019
0.3229895227008149 0.2730430733410943 0.4039674039580908
SouthBendElkhart 2019
0.3675689742319376 0.2587916184985562 0.37363940726950623
SouthBendElkhart 2019
0.32278933917488134 0.2876159182183279 0.3895947426067908
SouthBendElkhart 2019
0.32464511041009464 0.2734752891692955 0.4018796004206099
SouthBendElkhart 2019
0.40659260235073696 0.24635899237401016 0.3470484052752529
SouthBendElkhart 2019
0.35670405141452965 0.2690441686094134 0.37425177997605696
SouthBendElkhart 2019
0.33851333470551914 0.2617101355523538 0.3997765297421271
SouthBendElkhart 2019
0.34031797483985

Springfield 2019
0.4523998413328044 0.227356868967341 0.32024328969985455
Springfield 2019
0.5606817008090893 0.2731967636426235 0.16612153554828715
Springfield 2019
0.5035041611914148 0.27786903197547086 0.21862680683311433
Springfield 2019
0.5579406339091062 0.19537145732014086 0.24668790877075297
Springfield 2021
0.4915739347134151 0.23319516066684762 0.2752309046197373
Springfield 2021
0.5636150825338831 0.21433650196352977 0.22204841550258705
Springfield 2021
0.45132518721081843 0.2584951596340014 0.2901796531551802
Springfield 2021
0.44398525119065907 0.22599477646335842 0.33001997234598246
Springfield 2021
0.5083575225856952 0.19645477968755096 0.29518769772675385
Springfield 2021
0.4654594495508943 0.2656794196978029 0.2688611307513028
Springfield 2021
0.49012591720264514 0.27905607391973913 0.23081800887761572
Springfield 2021
0.6023898267923701 0.16888840166630126 0.22872177154132867
Springfield 2021
0.46609502618511384 0.20299568556155537 0.3309092882533308
Springfield 2021


0.13288406408593162 0.5085588551915047 0.35855708072256376
Tampa 2012
0.24978394883908508 0.5068790689635305 0.24333698219738434
Tampa 2012
0.16582965785495496 0.5514123462102468 0.2827579959347982
Tampa 2012
0.11035506763471409 0.5276559924239768 0.3619889399413092
Tampa 2012
0.20812366042307334 0.579023390177989 0.21285294939893765
Tampa 2012
0.16581503734067043 0.5947073309642797 0.23947763169504985
Tampa 2012
0.1252306633722203 0.5479936196165515 0.3267757170112282
Tampa 2016
0.19861168690590017 0.5106838023195889 0.290704510774511
Tampa 2016
0.28393552065173383 0.4781584883211739 0.23790599102709223
Tampa 2016
0.18397132986885126 0.5347876710057065 0.2812409991254423
Tampa 2016
0.12228260869565218 0.5151775115002226 0.3625398798041252
Tampa 2016
0.2168425375963604 0.47218448544589076 0.31097297695774884
Tampa 2016
0.1829244400908322 0.5277505677011443 0.2893249922080235
Tampa 2016
0.1742690184038103 0.5879056585543901 0.2378253230417995
Tampa 2016
0.3234001328375194 0.416509080755

0.3739259810303143 0.16198623767900316 0.4640877812906825
Waco 2021
0.42169686257180733 0.11577551922227132 0.46252761820592136
Waco 2021
0.35709954522486104 0.1707933299646286 0.47210712481051037
Waco 2021
0.3646204244409628 0.1892180601053981 0.4461615154536391
Waco 2021
0.3603420480829887 0.17039321511179645 0.46926473680521485
Waco 2021
0.36354491017964075 0.22941317365269462 0.40704191616766466
Waco 2021
0.347717945529163 0.18243999502549435 0.46984205944534263
WashingtonDC 2012
0.30491397045941565 0.3602327300211982 0.3348532995193862
WashingtonDC 2012
0.3328475315461015 0.34559961699759273 0.32155285145630574
WashingtonDC 2012
0.2717302664885488 0.3889665614237863 0.3393031720876649
WashingtonDC 2012
0.30979532098105855 0.33064932499472477 0.3595553540242167
WashingtonDC 2012
0.3433027136377366 0.2540364441218053 0.4026608422404581
WashingtonDC 2012
0.30287442715338597 0.35896927168601006 0.33815630116060397
WashingtonDC 2012
0.2784317142073302 0.4387498424373642 0.2828184433553

0.2554680441983831 0.25196321434362623 0.4925687414579907
WestPalmBeach 2021
0.21953153915296356 0.36534960835706354 0.4151188524899729
WestPalmBeach 2021
0.25732182455892355 0.23875111539107158 0.5039270600500049
WestPalmBeach 2021
0.262045377542556 0.26437033402264204 0.47358428843480194
WestPalmBeach 2021
0.23616662859753312 0.3511963225216994 0.41263704888076747
WestPalmBeach 2021
0.3112250101999184 0.15185978512171902 0.5369152046783626
WestPalmBeach 2021
0.26349389358613035 0.1908645771059299 0.5456415293079397
WestPalmBeach 2021
0.2202774922918808 0.32585817060637207 0.4538643371017472
WestPalmBeach 2021
0.29871556532026367 0.19952866505173417 0.5017557696280022
WestPalmBeach 2021
0.2678210063448108 0.2329778007698979 0.4992011928852913
WestPalmBeach 2021
0.21737215092894083 0.37002489944455086 0.41260294962650834
WestPalmBeach 2021
0.24938549663782741 0.30988294658422755 0.440731556777945
WestPalmBeach 2021
0.2382431011168428 0.3213301948214554 0.4404267040617018
WestPalmBeach 

In [9]:
def create_comparison_chart_json(csv_file_path, published_data_base_path = "./public"):
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        needsCaveat = row['NeedsCaveat']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
            "needsCaveat": needsCaveat
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts.json')
source_data_base_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie'
csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
create_comparison_chart_json(csv_file_path)

Albany-2012-1:All-1:All
Albany-2012-2:18-44-1:All
Albany-2012-3:45-64-1:All
Albany-2012-4:65+-1:All
Albany-2012-1:All-2:$0-50k
Albany-2012-1:All-3:$50-100k
Albany-2012-1:All-4:$100k+
Albany-2012-2:18-44-2:$0-50k
Albany-2012-3:45-64-2:$0-50k
Albany-2012-4:65+-2:$0-50k
Albany-2012-2:18-44-3:$50-100k
Albany-2012-3:45-64-3:$50-100k
Albany-2012-4:65+-3:$50-100k
Albany-2012-2:18-44-4:$100k+
Albany-2012-3:45-64-4:$100k+
Albany-2012-4:65+-4:$100k+
Albany-2016-1:All-1:All
Albany-2016-2:18-44-1:All
Albany-2016-3:45-64-1:All
Albany-2016-4:65+-1:All
Albany-2016-1:All-2:$0-50k
Albany-2016-1:All-3:$50-100k
Albany-2016-1:All-4:$100k+
Albany-2016-2:18-44-2:$0-50k
Albany-2016-3:45-64-2:$0-50k
Albany-2016-4:65+-2:$0-50k
Albany-2016-2:18-44-3:$50-100k
Albany-2016-3:45-64-3:$50-100k
Albany-2016-4:65+-3:$50-100k
Albany-2016-2:18-44-4:$100k+
Albany-2016-3:45-64-4:$100k+
Albany-2016-4:65+-4:$100k+
Albany-2019-1:All-1:All
Albany-2019-2:18-44-1:All
Albany-2019-3:45-64-1:All
Albany-2019-4:65+-1:All
Albany-2019-

Baton Rouge-2016-2:18-44-3:$50-100k
Baton Rouge-2016-3:45-64-3:$50-100k
Baton Rouge-2016-4:65+-3:$50-100k
Baton Rouge-2016-2:18-44-4:$100k+
Baton Rouge-2016-3:45-64-4:$100k+
Baton Rouge-2016-4:65+-4:$100k+
Baton Rouge-2019-1:All-1:All
Baton Rouge-2019-2:18-44-1:All
Baton Rouge-2019-3:45-64-1:All
Baton Rouge-2019-4:65+-1:All
Baton Rouge-2019-1:All-2:$0-50k
Baton Rouge-2019-1:All-3:$50-100k
Baton Rouge-2019-1:All-4:$100k+
Baton Rouge-2019-2:18-44-2:$0-50k
Baton Rouge-2019-3:45-64-2:$0-50k
Baton Rouge-2019-4:65+-2:$0-50k
Baton Rouge-2019-2:18-44-3:$50-100k
Baton Rouge-2019-3:45-64-3:$50-100k
Baton Rouge-2019-4:65+-3:$50-100k
Baton Rouge-2019-2:18-44-4:$100k+
Baton Rouge-2019-3:45-64-4:$100k+
Baton Rouge-2019-4:65+-4:$100k+
Baton Rouge-2021-1:All-1:All
Baton Rouge-2021-2:18-44-1:All
Baton Rouge-2021-3:45-64-1:All
Baton Rouge-2021-4:65+-1:All
Baton Rouge-2021-1:All-2:$0-50k
Baton Rouge-2021-1:All-3:$50-100k
Baton Rouge-2021-1:All-4:$100k+
Baton Rouge-2021-2:18-44-2:$0-50k
Baton Rouge-2021-3

Denver-2012-2:18-44-3:$50-100k
Denver-2012-3:45-64-3:$50-100k
Denver-2012-4:65+-3:$50-100k
Denver-2012-2:18-44-4:$100k+
Denver-2012-3:45-64-4:$100k+
Denver-2012-4:65+-4:$100k+
Denver-2016-1:All-1:All
Denver-2016-2:18-44-1:All
Denver-2016-3:45-64-1:All
Denver-2016-4:65+-1:All
Denver-2016-1:All-2:$0-50k
Denver-2016-1:All-3:$50-100k
Denver-2016-1:All-4:$100k+
Denver-2016-2:18-44-2:$0-50k
Denver-2016-3:45-64-2:$0-50k
Denver-2016-4:65+-2:$0-50k
Denver-2016-2:18-44-3:$50-100k
Denver-2016-3:45-64-3:$50-100k
Denver-2016-4:65+-3:$50-100k
Denver-2016-2:18-44-4:$100k+
Denver-2016-3:45-64-4:$100k+
Denver-2016-4:65+-4:$100k+
Denver-2019-1:All-1:All
Denver-2019-2:18-44-1:All
Denver-2019-3:45-64-1:All
Denver-2019-4:65+-1:All
Denver-2019-1:All-2:$0-50k
Denver-2019-1:All-3:$50-100k
Denver-2019-1:All-4:$100k+
Denver-2019-2:18-44-2:$0-50k
Denver-2019-3:45-64-2:$0-50k
Denver-2019-4:65+-2:$0-50k
Denver-2019-2:18-44-3:$50-100k
Denver-2019-3:45-64-3:$50-100k
Denver-2019-4:65+-3:$50-100k
Denver-2019-2:18-44-4

Jackson-2012-4:65+-1:All
Jackson-2012-1:All-2:$0-50k
Jackson-2012-1:All-3:$50-100k
Jackson-2012-1:All-4:$100k+
Jackson-2012-2:18-44-2:$0-50k
Jackson-2012-3:45-64-2:$0-50k
Jackson-2012-4:65+-2:$0-50k
Jackson-2012-2:18-44-3:$50-100k
Jackson-2012-3:45-64-3:$50-100k
Jackson-2012-4:65+-3:$50-100k
Jackson-2012-2:18-44-4:$100k+
Jackson-2012-3:45-64-4:$100k+
Jackson-2012-4:65+-4:$100k+
Jackson-2016-1:All-1:All
Jackson-2016-2:18-44-1:All
Jackson-2016-3:45-64-1:All
Jackson-2016-4:65+-1:All
Jackson-2016-1:All-2:$0-50k
Jackson-2016-1:All-3:$50-100k
Jackson-2016-1:All-4:$100k+
Jackson-2016-2:18-44-2:$0-50k
Jackson-2016-3:45-64-2:$0-50k
Jackson-2016-4:65+-2:$0-50k
Jackson-2016-2:18-44-3:$50-100k
Jackson-2016-3:45-64-3:$50-100k
Jackson-2016-4:65+-3:$50-100k
Jackson-2016-2:18-44-4:$100k+
Jackson-2016-3:45-64-4:$100k+
Jackson-2016-4:65+-4:$100k+
Jackson-2019-1:All-1:All
Jackson-2019-2:18-44-1:All
Jackson-2019-3:45-64-1:All
Jackson-2019-4:65+-1:All
Jackson-2019-1:All-2:$0-50k
Jackson-2019-1:All-3:$50-10

LittleRock-2021-1:All-1:All
LittleRock-2021-2:18-44-1:All
LittleRock-2021-3:45-64-1:All
LittleRock-2021-4:65+-1:All
LittleRock-2021-1:All-2:$0-50k
LittleRock-2021-1:All-3:$50-100k
LittleRock-2021-1:All-4:$100k+
LittleRock-2021-2:18-44-2:$0-50k
LittleRock-2021-3:45-64-2:$0-50k
LittleRock-2021-4:65+-2:$0-50k
LittleRock-2021-2:18-44-3:$50-100k
LittleRock-2021-3:45-64-3:$50-100k
LittleRock-2021-4:65+-3:$50-100k
LittleRock-2021-2:18-44-4:$100k+
LittleRock-2021-3:45-64-4:$100k+
LittleRock-2021-4:65+-4:$100k+
LosAngeles-2012-1:All-1:All
LosAngeles-2012-2:18-44-1:All
LosAngeles-2012-3:45-64-1:All
LosAngeles-2012-4:65+-1:All
LosAngeles-2012-1:All-2:$0-50k
LosAngeles-2012-1:All-3:$50-100k
LosAngeles-2012-1:All-4:$100k+
LosAngeles-2012-2:18-44-2:$0-50k
LosAngeles-2012-3:45-64-2:$0-50k
LosAngeles-2012-4:65+-2:$0-50k
LosAngeles-2012-2:18-44-3:$50-100k
LosAngeles-2012-3:45-64-3:$50-100k
LosAngeles-2012-4:65+-3:$50-100k
LosAngeles-2012-2:18-44-4:$100k+
LosAngeles-2012-3:45-64-4:$100k+
LosAngeles-2012

NewOrleans-2012-1:All-4:$100k+
NewOrleans-2012-2:18-44-2:$0-50k
NewOrleans-2012-3:45-64-2:$0-50k
NewOrleans-2012-4:65+-2:$0-50k
NewOrleans-2012-2:18-44-3:$50-100k
NewOrleans-2012-3:45-64-3:$50-100k
NewOrleans-2012-4:65+-3:$50-100k
NewOrleans-2012-2:18-44-4:$100k+
NewOrleans-2012-3:45-64-4:$100k+
NewOrleans-2012-4:65+-4:$100k+
NewOrleans-2016-1:All-1:All
NewOrleans-2016-2:18-44-1:All
NewOrleans-2016-3:45-64-1:All
NewOrleans-2016-4:65+-1:All
NewOrleans-2016-1:All-2:$0-50k
NewOrleans-2016-1:All-3:$50-100k
NewOrleans-2016-1:All-4:$100k+
NewOrleans-2016-2:18-44-2:$0-50k
NewOrleans-2016-3:45-64-2:$0-50k
NewOrleans-2016-4:65+-2:$0-50k
NewOrleans-2016-2:18-44-3:$50-100k
NewOrleans-2016-3:45-64-3:$50-100k
NewOrleans-2016-4:65+-3:$50-100k
NewOrleans-2016-2:18-44-4:$100k+
NewOrleans-2016-3:45-64-4:$100k+
NewOrleans-2016-4:65+-4:$100k+
NewOrleans-2019-1:All-1:All
NewOrleans-2019-2:18-44-1:All
NewOrleans-2019-3:45-64-1:All
NewOrleans-2019-4:65+-1:All
NewOrleans-2019-1:All-2:$0-50k
NewOrleans-2019-1

Omaha-2012-4:65+-4:$100k+
Omaha-2016-1:All-1:All
Omaha-2016-2:18-44-1:All
Omaha-2016-3:45-64-1:All
Omaha-2016-4:65+-1:All
Omaha-2016-1:All-2:$0-50k
Omaha-2016-1:All-3:$50-100k
Omaha-2016-1:All-4:$100k+
Omaha-2016-2:18-44-2:$0-50k
Omaha-2016-3:45-64-2:$0-50k
Omaha-2016-4:65+-2:$0-50k
Omaha-2016-2:18-44-3:$50-100k
Omaha-2016-3:45-64-3:$50-100k
Omaha-2016-4:65+-3:$50-100k
Omaha-2016-2:18-44-4:$100k+
Omaha-2016-3:45-64-4:$100k+
Omaha-2016-4:65+-4:$100k+
Omaha-2019-1:All-1:All
Omaha-2019-2:18-44-1:All
Omaha-2019-3:45-64-1:All
Omaha-2019-4:65+-1:All
Omaha-2019-1:All-2:$0-50k
Omaha-2019-1:All-3:$50-100k
Omaha-2019-1:All-4:$100k+
Omaha-2019-2:18-44-2:$0-50k
Omaha-2019-3:45-64-2:$0-50k
Omaha-2019-4:65+-2:$0-50k
Omaha-2019-2:18-44-3:$50-100k
Omaha-2019-3:45-64-3:$50-100k
Omaha-2019-4:65+-3:$50-100k
Omaha-2019-2:18-44-4:$100k+
Omaha-2019-3:45-64-4:$100k+
Omaha-2019-4:65+-4:$100k+
Omaha-2021-1:All-1:All
Omaha-2021-2:18-44-1:All
Omaha-2021-3:45-64-1:All
Omaha-2021-4:65+-1:All
Omaha-2021-1:All-2:$0-

SanFranciscoOaklandSanJose-2019-1:All-3:$50-100k
SanFranciscoOaklandSanJose-2019-1:All-4:$100k+
SanFranciscoOaklandSanJose-2019-2:18-44-2:$0-50k
SanFranciscoOaklandSanJose-2019-3:45-64-2:$0-50k
SanFranciscoOaklandSanJose-2019-4:65+-2:$0-50k
SanFranciscoOaklandSanJose-2019-2:18-44-3:$50-100k
SanFranciscoOaklandSanJose-2019-3:45-64-3:$50-100k
SanFranciscoOaklandSanJose-2019-4:65+-3:$50-100k
SanFranciscoOaklandSanJose-2019-2:18-44-4:$100k+
SanFranciscoOaklandSanJose-2019-3:45-64-4:$100k+
SanFranciscoOaklandSanJose-2019-4:65+-4:$100k+
SanFranciscoOaklandSanJose-2021-1:All-1:All
SanFranciscoOaklandSanJose-2021-2:18-44-1:All
SanFranciscoOaklandSanJose-2021-3:45-64-1:All
SanFranciscoOaklandSanJose-2021-4:65+-1:All
SanFranciscoOaklandSanJose-2021-1:All-2:$0-50k
SanFranciscoOaklandSanJose-2021-1:All-3:$50-100k
SanFranciscoOaklandSanJose-2021-1:All-4:$100k+
SanFranciscoOaklandSanJose-2021-2:18-44-2:$0-50k
SanFranciscoOaklandSanJose-2021-3:45-64-2:$0-50k
SanFranciscoOaklandSanJose-2021-4:65+-2:$0

Syracuse-2012-4:65+-1:All
Syracuse-2012-1:All-2:$0-50k
Syracuse-2012-1:All-3:$50-100k
Syracuse-2012-1:All-4:$100k+
Syracuse-2012-2:18-44-2:$0-50k
Syracuse-2012-3:45-64-2:$0-50k
Syracuse-2012-4:65+-2:$0-50k
Syracuse-2012-2:18-44-3:$50-100k
Syracuse-2012-3:45-64-3:$50-100k
Syracuse-2012-4:65+-3:$50-100k
Syracuse-2012-2:18-44-4:$100k+
Syracuse-2012-3:45-64-4:$100k+
Syracuse-2012-4:65+-4:$100k+
Syracuse-2016-1:All-1:All
Syracuse-2016-2:18-44-1:All
Syracuse-2016-3:45-64-1:All
Syracuse-2016-4:65+-1:All
Syracuse-2016-1:All-2:$0-50k
Syracuse-2016-1:All-3:$50-100k
Syracuse-2016-1:All-4:$100k+
Syracuse-2016-2:18-44-2:$0-50k
Syracuse-2016-3:45-64-2:$0-50k
Syracuse-2016-4:65+-2:$0-50k
Syracuse-2016-2:18-44-3:$50-100k
Syracuse-2016-3:45-64-3:$50-100k
Syracuse-2016-4:65+-3:$50-100k
Syracuse-2016-2:18-44-4:$100k+
Syracuse-2016-3:45-64-4:$100k+
Syracuse-2016-4:65+-4:$100k+
Syracuse-2019-1:All-1:All
Syracuse-2019-2:18-44-1:All
Syracuse-2019-3:45-64-1:All
Syracuse-2019-4:65+-1:All
Syracuse-2019-1:All-2:

Wrote market summary charts data to  ./public/market_comparison_charts.json


In [10]:
def create_data_options_json(csv_file_path, published_data_base_path = "./public"):
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
            "needsCaveat": row["NeedsCaveat"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options.json')
source_data_base_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie'
csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
create_data_options_json(csv_file_path)

Wrote data options data to  ./public/data_options.json


/var/folders/dh/fn0cs6ln6sq0722wd5lcgdcm0000gn/T/ipykernel_80479/2086740279.py:28: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
